In [ ]:
# load image
from func.utils import get_model_output_id_wnid_class_dict # get mapping: format: {"Model Ouput ID": ["WNID", "Class"]}
from func.utils import get_imagenet_id_wnid_class_dict # get mapping: format: {"ImageNet ID": ["WNID", "class"]}, e.g. {..."233": ['n02106382', 'Bouvier_des_Flandres'], ...}
from func.utils import map_model_id_to_imagenet_id, map_imagenet_id_to_model_id # mapping funcs

from func.utils import save_obj, load_obj

from func.concept_classifier import get_linear_classifier, get_xgb_classifier, prediction, batch

from func.responsible_regions import load_responsible_regions_from_given_path, X_y_preparation, process_cat_saliency_map

from func.show import load_feature_saliency_map_and_resized_img_for_show, show_concept_region_on_img

import numpy as np
import matplotlib.pyplot as plt
import copy
import os
import gc
import shap
import xgboost as xgb
from xgboost import XGBClassifier
import pandas as pd

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
# get the dict of ImageNet ID, WNID and class name
# format: {"ImageNet ID": ["WNID", "class"]}, e.g. {..."233": ['n02106382', 'Bouvier_des_Flandres'], ...}
imagenet_id_label=get_imagenet_id_wnid_class_dict(matfilepath = "imagenet_info/ILSVRC2012_meta.mat")

# get the dict of model output ID, WNID and class name
# format: {"Model Ouput ID": ["WNID", "Class"]}
modeloutput_id_label=get_model_output_id_wnid_class_dict(jsonfilepath = "imagenet_info/imagenet_label_index.json")

# get dict map model output ID to ImageNet ID
map_dict_model2imagenet=map_model_id_to_imagenet_id(imagenet_id_label, modeloutput_id_label)

# get ImageNet ID to dict map model output ID
map_dict_imagenet2model=map_imagenet_id_to_model_id(imagenet_id_label, modeloutput_id_label)

### show imagenet classes

In [ ]:
# only 1~1000 is valid
for idx in imagenet_id_label:
    print(str(idx)+": "+str(imagenet_id_label[idx]))

### imagenet parent and child

In [ ]:
imagenet_class_parent_and_child_dict = load_obj("imagenet_info/imagenet_class_dict")

In [ ]:
# e.g.
imagenet_class_parent_and_child_dict[1095]

### Concept classifier and Shapley values

In [ ]:
# set the path to responsible regions
responisble_regions_save_path = "output/responsible_regions_vgg19"

In [ ]:
# under that path, there are responsible regions of diff layers...
print(os.listdir(responisble_regions_save_path))

In [ ]:
chosen_layer = "features.30"
print("chosen layer: "+chosen_layer)

In [ ]:
# there are diff ID groups...
print(os.listdir(os.path.join(responisble_regions_save_path, chosen_layer)))

In [ ]:
chosen_ID_group = "[1082, 1095, 1845]"
print("chosen_ID_group: "+chosen_ID_group)

In [ ]:
# load data
X_y = np.load(os.path.join(responisble_regions_save_path, chosen_layer, chosen_ID_group, "X_y.npz"))

In [ ]:
# set the target obj (to the the concept classifier of the obj)
target_obj_loc = 1

In [ ]:
y = copy.deepcopy(X_y["y"])
print("target is "+str(chosen_ID_group.split(",")[target_obj_loc]))
print(imagenet_class_parent_and_child_dict[int(chosen_ID_group.split(",")[target_obj_loc])]['words'])
print("we will train a concept classifier to distinguish the target from the others in "+str(chosen_ID_group))
y[y!=target_obj_loc] = -1
y[y==target_obj_loc] = 1
y[y!=1] = 0

In [ ]:
# train the concept classifier
coef, clf, Validation_score = get_linear_classifier(X_y["X"], y, classifier = "SGD")

In [ ]:
pred_of_pos_input = clf.predict(X_y["X"][y==1])
pred_of_neg_input = clf.predict(X_y["X"][y==0])

precision = sum(pred_of_pos_input) / len(pred_of_pos_input)
recall = sum(pred_of_pos_input) / (sum(pred_of_pos_input)+sum(pred_of_neg_input))
f1 = 2 * precision * recall / (precision + recall)
print("precision: "+str(precision))
print("recall: "+str(recall))
print("f1: "+str(f1))

In [ ]:
need_to_save_the_clf = False

clf_save_path = "output/concept_clf"
if not os.path.exists(clf_save_path):
    os.makedirs(clf_save_path)

clf_name = "concept_clf"

clf_dict = {}
clf_dict["clf"] = clf
clf_dict["info"] = {"name": "classifier", "model": "vgg19", "layer": chosen_layer}

if need_to_save_the_clf:
    save_obj(clf_dict, os.path.join(clf_save_path, clf_name))

train xgb_classifier and calculate Shapley Values with GPU

In [ ]:
bst = get_xgb_classifier(X_y["X"],y)
bst.set_param({"predictor": "gpu_predictor"})
explainer = shap.TreeExplainer(bst)
shap_values = explainer.shap_values(X_y["X"])

In [ ]:
# # if the size of X is too large...
# # use the following code

# X = copy.deepcopy(X_y["X"])
# ROUNDS = 5
# K = 4
# flag = True
# for ROUND in range(ROUNDS):
#     row_num = np.arange(X.shape[0])
#     np.random.shuffle(row_num)
#     X_shuffle = X[row_num, :]
#     y_shuffle = y[row_num]
#     batcherator = batch(X_shuffle, y_shuffle, int(X.shape[0] / K))
#     for index, (chunk_X, chunk_y) in enumerate(batcherator):
#         if len(chunk_X) == int(X.shape[0] / K):
#             current_batch_size = len(chunk_X)
#             print("batch size: "+str(current_batch_size)+" | "+str(index)+"th batch of "+str(ROUND)+"th round", end="\r")
#             if flag:
#                 bst = get_xgb_classifier(chunk_X, chunk_y, model_para = False)
#                 flag = False
#             else:
#                 bst = get_xgb_classifier(chunk_X, chunk_y, model_para = True)
        
# bst.set_param({"predictor": "gpu_predictor"})
# explainer = shap.TreeExplainer(bst)
        
# batcherator_for_shap = batch(X, y, current_batch_size)
# print("the batch size of shap cal is "+str(current_batch_size))
# for index, (chunk_X, chunk_y) in enumerate(batcherator_for_shap):
#     if len(chunk_X) == current_batch_size:
#         if index == 0:
#             shap_values = explainer.shap_values(chunk_X)
#             #dmat = xgb.DMatrix(data=chunk_X,label=chunk_y)
#             #shap_values = bst.predict(dmat, pred_contribs=True)
        
#         else:
#             shap_values = np.concatenate((shap_values, explainer.shap_values(chunk_X)))
#             #dmat = xgb.DMatrix(data=chunk_X,label=chunk_y)
#             #shap_values = np.concatenate((shap_values, bst.predict(dmat, pred_contribs=True)))

In [ ]:
importance_arr_mean = []
importance_arr_median = []
importance_arr_max = []
importance_arr_min = []

for i in range(shap_values.shape[1]):
    shap_vals_of_one_channel = np.abs(shap_values[:,i]) #xgb_shap_values[:,i]
    importance_arr_mean.append(np.mean(shap_vals_of_one_channel))
    importance_arr_median.append(np.median(shap_vals_of_one_channel))
    importance_arr_max.append(np.max(shap_vals_of_one_channel))
    importance_arr_min.append(np.min(shap_vals_of_one_channel))

importance_arr_mean = np.array(importance_arr_mean)
importance_arr_median = np.array(importance_arr_median)
importance_arr_max = np.array(importance_arr_max)
importance_arr_min = np.array(importance_arr_min)

In [ ]:
# plot shap vals

sort_locs = np.argsort(importance_arr_mean)[::-1]

sort_locs_str = []
for i in sort_locs:
     sort_locs_str.append(str(i))

N_subplot = 7
len_of_one_sub = int(len(sort_locs)/N_subplot+1)
sort_locs_sub = []
sort_locs_str_sub = []
for i in range(N_subplot):
    start_loc = np.clip(i*len_of_one_sub,0,len(sort_locs))
    end_loc = np.clip((i+1)*len_of_one_sub,0,len(sort_locs))
    sort_locs_sub.append(sort_locs[start_loc:end_loc])
    sort_locs_str_sub.append(sort_locs_str[start_loc:end_loc])
    
fig, axs = plt.subplots(1,N_subplot,figsize=(20,20))

for i in range(N_subplot):
    axs[i].barh(sort_locs_str_sub[i][::-1],importance_arr_mean[sort_locs_sub[i]][::-1])
    axs[i].set_xlim(0, np.max(importance_arr_mean)+0.05)
    axs[i].set_xlabel('Shapley')
# plt.savefig("shap_bar_plot.svg",
#             bbox_inches='tight', dpi=100, pad_inches=0.0)

In [ ]:
# save shap vals

need_to_save_the_shap_vals = False
shap_vals_save_path = "output"
if need_to_save_the_shap_vals:
    shap_df = pd.DataFrame(columns=['mean', 'median', 'max', 'min'])
    shap_df['mean'] = importance_arr_mean
    shap_df['median'] = importance_arr_median
    shap_df['max'] = importance_arr_max
    shap_df['min'] = importance_arr_min
    shap_df.to_csv(os.path.join(shap_vals_save_path, "shap_values.csv"))

### Show the results of concept classifier

In [ ]:
# set the path to load feature maps and saliency maps of a model
feature_maps_and_saliency_maps_path = "output/feature_maps_and_saliency_maps_vgg19"

ID_for_show = 1095

In [ ]:
print(ID_for_show, imagenet_class_parent_and_child_dict[ID_for_show]['words'])

resized_img_for_show_cat, feature_map_for_show_cat, saliency_map_for_show_cat = \
load_feature_saliency_map_and_resized_img_for_show(feature_maps_and_saliency_maps_path+"/result_of_ID_"+str(ID_for_show), chosen_layer)

resized_img_for_show_cat = np.array(resized_img_for_show_cat, dtype=int)

grayscale_saliency_map = process_cat_saliency_map(saliency_map_for_show_cat, num_of_pic_of_a_row = 10, mode="norm")

concept_map = prediction(clf, feature_map_for_show_cat, is_predict_proba = True)

In [ ]:
show_concept_region_on_img(resized_img_for_show_cat, concept_map[:,:,1])